In [6]:
import os  # for file paths
import pandas as pd
import awswrangler as wr
import pydbtools as pydb  # see https://github.com/moj-analytical-services/pydbtools
import altair as alt  # for plotting, see https://altair-viz.github.io/getting_started/overview.html

# few things for viewing dataframes better
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [11]:
#Applications by type - children involved
ac = f"""
SELECT
  Year,
  Quarter,
  'Application' AS Type,
  'Children' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
   Order_desc AS Order_type,
   Order_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  {database2}.ca_apps_child
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  Case_type,
  Order_desc,
  Order_code
"""

pydb.create_temp_table(ac, "ca_apps_type_child")

In [16]:
#Orders applied for (by order type)
ao = f"""
SELECT
  Year,
  Quarter,
  'Application' AS Type,
  'Order type' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
   Order_desc,
   Order_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_apps_order_count
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  Case_type,
  Order_desc,
  Order_code
"""

pydb.create_temp_table(ao, "ca_apps_order_type")

In [19]:
#Application events
ae = f"""
SELECT
  Year,
  Quarter,
  'Application' AS Type,
  'Application events' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
   'All' AS Order_type,
   ' ' AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_apps_event_count
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  Case_type
"""

pydb.create_temp_table(ae, "ca_apps_events")

In [31]:
#Case starts - including number of applicants/respondents and Hight court indicator*/
ap = f"""
SELECT
  P.Year,
  P.Quarter,
  'Cases' AS Type,
  'Cases starting' AS Count_type,
  CASE WHEN P.Case_type = 'C' THEN 'Public law'
       WHEN P.Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
  'All' AS Order_type,
   ' ' AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
   Case When P.no_of_applicants = 1 THEN '1'
      ELSE '2+'
    END AS Applicants_in_case,
   Case When P.no_of_respondents in (1,2,3)
         THEN CAST(P.no_of_respondents AS VARCHAR)
         ELSE '4+'
        END AS Respondents_in_case,
   CASE WHEN HC.case_hc_value IS NULL THEN 'No'
          ELSE HC.case_hc_value
     END AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.party_count P
  left join __temp__.ca_cases_HC HC
    on P.Case_number = HC.Case_number
     and P.case_type = HC.case_type
Where
  P.Year > 2010
Group BY
  P.Year,
  P.Quarter,
  p.case_type,
  Case When P.no_of_applicants = 1 THEN '1'
      ELSE '2+'
    END,
  Case When P.no_of_respondents in (1,2,3)
         THEN CAST(P.no_of_respondents AS VARCHAR)
         ELSE '4+'
        END,      
  CASE WHEN HC.case_hc_value IS NULL THEN 'No'
          ELSE HC.case_hc_value
     END
"""

pydb.create_temp_table(ap, "ca_apps_cases")

In [30]:
#Individual children - annual - including age of child
ci = f"""
SELECT
  Year,
  ' ' AS Quarter,
  'Application' AS Type,
  'Individual children' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
  'All' AS Order_type,
  ' ' AS Order_type_code,
  Gender,
  age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
  COUNT(*) AS Count
FROM
  childout.APP_CHILD_IND_COUNT_DATA_YR
Where
  Year > 2010
Group BY
  Year,
  Public_private,
  Gender2,
  age_band
"""

pydb.create_temp_table(ci, "ca_apps_ind_yr")